In [3]:
import cv2
import numpy as np
from numba import cuda
import os
import glob

def simulate_daltonism(image, daltonism_type):
    image_copy = image.copy()
    if daltonism_type == "Deutéranopie":
        image_copy[:, :, 1] = 0
    return image_copy

@cuda.jit
def analyser_pixels_kernel(image, rouge_count, vert_count, bleu_count, daltonien_count):
    x, y = cuda.grid(2)
    if x < image.shape[0] and y < image.shape[1]:
        pixel = image[x, y]
        bleu, vert, rouge = pixel
        if rouge > vert and rouge > bleu:
            cuda.atomic.add(rouge_count, 0, 1)
        elif vert > rouge and vert > bleu:
            cuda.atomic.add(vert_count, 0, 1)
            cuda.atomic.add(daltonien_count, 0, 1)
        elif bleu > rouge and bleu > vert:
            cuda.atomic.add(bleu_count, 0, 1)

def traiter_image(image, chemin_image):
    rouge_count = np.zeros(1, dtype=np.int32)
    vert_count = np.zeros(1, dtype=np.int32)
    bleu_count = np.zeros(1, dtype=np.int32)
    daltonien_count = np.zeros(1, dtype=np.int32)

    image_gpu = cuda.to_device(image)
    threadsperblock = (16, 16)
    blockspergrid = (
        (image.shape[0] + threadsperblock[0] - 1) // threadsperblock[0],
        (image.shape[1] + threadsperblock[1] - 1) // threadsperblock[1]
    )

    start = cuda.event(timing=True)
    stop = cuda.event(timing=True)
    start.record()

    analyser_pixels_kernel[blockspergrid, threadsperblock](image_gpu,
        rouge_count, vert_count, bleu_count, daltonien_count)

    stop.record()
    stop.synchronize()

    rouge = rouge_count[0]
    vert = vert_count[0]
    bleu = bleu_count[0]
    daltonien = daltonien_count[0]
    total_pixels = image.shape[0] * image.shape[1]

    daltonien_percentage = (daltonien / total_pixels) * 100

    nom_image = os.path.basename(chemin_image)

    if daltonien_percentage > 5:
        print(f"{nom_image} non visible par les daltoniens ({daltonien_percentage:.2f}%)")
        output_folder = "output/images_non_visibles_par_daltoniens_transformé"
        os.makedirs(output_folder, exist_ok=True)
        modified = simulate_daltonism(image, "Deutéranopie")
        cv2.imwrite(os.path.join(output_folder, f"deuteranopie_{nom_image}"), modified)
    else:
        print(f"{nom_image} visible par les daltoniens ({daltonien_percentage:.2f}%)")
        output_folder = "output/images_visibles_par_daltoniens_de_base"
        os.makedirs(output_folder, exist_ok=True)
        cv2.imwrite(os.path.join(output_folder, nom_image), image)

    print(f"Temps CUDA : {cuda.event_elapsed_time(start, stop)/1000:.4f} sec")

def main():
    dossier_images = "Input"
    chemins_images = glob.glob(os.path.join(dossier_images, "*.jpg"))

    if not chemins_images:
        print("Aucun fichier .jpg trouvé dans le dossier Input/")
        return
    
    start_total = cuda.event(timing=True)
    stop_total = cuda.event(timing=True)
    start_total.record()
    
    for chemin in chemins_images:
        print(f"\nTraitement : {chemin}")
        image = cv2.imread(chemin)
        if image is None:
            print(f"Erreur de lecture de l'image {chemin}")
            continue
        traiter_image(image, chemin)

    stop_total.record()
    stop_total.synchronize()

    total_time_ms = cuda.event_elapsed_time(start_total, stop_total)
    print(f"\nTemps total CUDA pour toutes les images : {total_time_ms/1000:.2f} secondes")

if __name__ == "__main__":
    main()


Traitement : Input/2_deut.jpg
2_deut.jpg non visible par les daltoniens (10.96%)
Temps CUDA : 0.0774 sec

Traitement : Input/4_deut.jpg
4_deut.jpg non visible par les daltoniens (26.47%)
Temps CUDA : 0.0011 sec

Traitement : Input/8_visible.jpg
8_visible.jpg visible par les daltoniens (4.29%)
Temps CUDA : 0.0011 sec

Traitement : Input/3_deut.jpg
3_deut.jpg non visible par les daltoniens (30.84%)
Temps CUDA : 0.0014 sec

Traitement : Input/1_deut.jpg
1_deut.jpg non visible par les daltoniens (48.85%)
Temps CUDA : 0.0010 sec

Temps total CUDA pour toutes les images : 0.13 secondes


/opt/conda/lib/python3.12/site-packages/numba/cuda/cudadrv/devicearray.py:887: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [2]:
#pip install opencv-python